
# TradingView Strategy Backtest Playground

Notebook ini memuat pipeline lengkap untuk menguji sinyal strategi yang diekspor dari TradingView. Ia memandu mulai dari memuat data CSV, mengadaptasikannya ke format QF-Lib, menjalankan backtest, hingga analisis trade kalah dan eksperimen optimasi parameter tambahan.



> **Struktur notebook**
>
> 1. Parameter input & pemuatan data
> 2. Adaptasi data ke QF-Lib
> 3. Strategi berbasis sinyal
> 4. Menjalankan backtest & mengekstrak hasil
> 5. Visualisasi
> 6. Analisis trade kalah & investigasi
> 7. Eksplor optimasi parameter
> 8. Dokumentasi & reusable structure


In [1]:

from __future__ import annotations

import sys
from pathlib import Path
from typing import Dict, Iterable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.switch_backend('Agg')

plt.style.use("seaborn-v0_8-darkgrid")

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
if str(PROJECT_ROOT / 'src') not in sys.path:
    sys.path.append(str(PROJECT_ROOT / 'src'))

from src.strategy_backtest import (
    SignalBacktester,
    get_strategy,
    list_strategies,
    load_strategy_csv,
)


## 1. Parameter input & pemuatan data

In [2]:

DATA_FILE = PROJECT_ROOT / 'data' / 'OKX_BTCUSDT, 1D.csv'
TIME_COLUMN = 'time'
PRICE_COLUMN = 'close'
ASSET_SYMBOL = 'BTCUSDT'

STRATEGY_NAME = 'ema112_atr'
STRATEGY_KWARGS = {
    'fast_span': 50,
    'slow_span': 112,
    'atr_window': 14,
    'atr_multiplier': 1.5,
}

OPTIMISATION_GRID = {
    'atr_multiplier': [1.0, 1.5, 2.0],
}


In [3]:

data, column_mapping = load_strategy_csv(DATA_FILE, time_column=TIME_COLUMN)
print(f'Dataset berisi {len(data):,} bar dengan {len(data.columns)} kolom.')
print('Contoh mapping kolom (sanitised -> original):')
for alias, original in list(column_mapping.items())[:10]:
    print(f'  {alias} -> {original}')

data.head()


Dataset berisi 300 bar dengan 48 kolom.
Contoh mapping kolom (sanitised -> original):
  time -> time
  open -> open
  high -> high
  low -> low
  close -> close
  ema -> EMA
  lucid_connector -> LUCID Connector
  internal_higher_high -> Internal Higher High
  internal_lower_high -> Internal Lower High
  internal_lower_low -> Internal Lower Low


,open,high,low,close,ema,lucid_connector,internal_higher_high,internal_lower_high,internal_lower_low,internal_higher_low,...,upper_confluence_zone,lower_confluence_zone,confluence_meter_value,custom_alert_condition_highlighter_1,alert_scripting_condition_highlighter_1,at_valuewhen_1,atr,histogram,macd,signal
time,,,,,,,,,,,,,,,,,,,,,
2025-01-17,99990.9,105847.4,99950.1,104084.4,87477.819959,2201170739202,NaN,NaN,NaN,NaN,...,105,-5,71.428571,NaN,NaN,NaN,3858.413818,761.135818,806.234733,45.098916
2025-01-18,104084.5,104984.0,102288.0,104551.6,87780.010756,524290,NaN,NaN,NaN,NaN,...,105,-5,71.428571,NaN,NaN,NaN,3775.384260,974.775303,1263.568044,288.792741
2025-01-19,104551.6,106448.6,99639.4,101334.0,88019.904371,40962,NaN,NaN,NaN,NaN,...,105,-5,71.428571,NaN,NaN,NaN,3992.085384,849.608478,1350.803339,501.194861
2025-01-20,101334.0,109800.0,99526.4,102264.3,88272.017568,8194,109800.0,NaN,NaN,NaN,...,105,-5,57.142857,NaN,NaN,NaN,4440.765000,781.418731,1477.968275,696.549544
2025-01-21,102263.9,107285.8,100111.0,106150.1,88588.443806,67586,NaN,NaN,NaN,NaN,...,105,-5,57.142857,NaN,NaN,NaN,4636.053214,939.347827,1870.734327,931.386500



## 2. Adaptasi data ke QF-Lib

Data sudah menggunakan `DatetimeIndex` dan kolom harga numerik sehingga siap dipakai sebagai sumber harga untuk QF-Lib. Keluaran sanitasi kolom memudahkan pemanggilan strategi Python di langkah berikutnya.



## 3. Strategi berbasis sinyal

Strategi yang dipakai saat ini adalah **EMA 112 dengan exit ATR**:

- **Entry**: ketika EMA cepat (default 50) menyilang ke atas EMA lambat (default 112).
- **Exit**: ketika harga penutupan turun di bawah garis trailing stop, yaitu EMA lambat dikurangi ATR * multiplier.
- **Parameter**: periode EMA dan konfigurasi ATR dapat diubah dari sel parameter di atas.

Strategi tambahan dapat ditaruh dalam folder `src/strategy_backtest/strategies/` dan otomatis terdaftar menggunakan `registry`.


In [4]:

available = list_strategies()
print('Strategi tersedia:', ', '.join(available))
strategy = get_strategy(STRATEGY_NAME, **STRATEGY_KWARGS)
print('Deskripsi strategi:')
print(f"- Nama: {strategy.metadata.name}")
print(f"- Deskripsi: {strategy.metadata.description}")
print(f"- Entry: {strategy.metadata.entry}")
print(f"- Exit: {strategy.metadata.exit}")
print(f"- Parameter default: {strategy.metadata.parameters}")


Strategi tersedia: ema112_atr
Deskripsi strategi:
- Nama: ema112_atr
- Deskripsi: Strategi trend-following yang menggunakan EMA 50 dan EMA 112 sebagai filter arah dan trailing stop berbasis ATR untuk manajemen risiko.
- Entry: Entry long ketika EMA fast (50) menyilang ke atas EMA slow (112). Sinyal ini menandakan perubahan tren ke arah bullish.
- Exit: Exit long ketika harga penutupan jatuh di bawah EMA slow dikurangi ATR * multiplier, memberikan ruang bernapas sambil melindungi profit.
- Parameter default: {'fast_span': 50, 'slow_span': 112, 'atr_window': 14, 'atr_multiplier': 1.5}


In [5]:

signals = strategy.generate_signals(data)
print('Kolom sinyal:', list(signals.columns))
signals.head()


Kolom sinyal: ['long_entry', 'long_exit', 'short_entry', 'short_exit', 'ema_fast', 'ema_slow', 'atr', 'atr_trailing_stop']


,long_entry,long_exit,short_entry,short_exit,ema_fast,ema_slow,atr,atr_trailing_stop
time,,,,,,,,
2025-01-17,False,False,False,False,104084.400000,104084.400000,5897.300000,95238.450000
2025-01-18,True,False,False,False,104102.721569,104092.669027,4296.650000,97647.694027
2025-01-19,False,False,False,False,103994.144252,104043.843026,5134.166667,96342.593026
2025-01-20,False,False,False,False,103926.307223,104012.346689,6419.025000,94383.809189
2025-01-21,False,False,False,False,104013.514783,104050.183031,6570.180000,94194.913031


## 4. Menjalankan backtest & mengekstrak hasil

In [6]:

backtester = SignalBacktester(data=data, price_column=PRICE_COLUMN)
outputs = backtester.run(signals)

metrics = outputs.metrics
trade_summary = outputs.trade_summary
print('Metrik performa (QF-Lib / fallback):')
for key, value in metrics.items():
    if isinstance(value, (int, float, np.floating)):
        print(f'- {key}: {value:.4f}')
    else:
        print(f'- {key}: {value}')

print('Ringkasan trade:')
for key, value in trade_summary.items():
    print(f'- {key}: {value}')


Metrik performa (QF-Lib / fallback):
- total_return: -0.0541
- cagr: -0.0456
- sharpe_ratio: -0.0952
- annualised_vol: 0.2247
- max_drawdown: -0.1463
- avg_drawdown_duration: 41.1429
Ringkasan trade:
- total_trades: 2
- long_trades: 2
- short_trades: 0
- win_rate: 0.5
- avg_pnl_pct: -0.02615406781256291
- median_bars: 96.0


In [7]:
outputs.trades.head()

,trade_id,direction,entry_time,exit_time,entry_price,exit_price,pnl_pct,pnl_currency,bars_held,exit_reason,entry_ema_fast,entry_ema_slow,entry_atr,entry_atr_trailing_stop,exit_ema_fast,exit_ema_slow,exit_atr,exit_atr_trailing_stop
0,1,Long,2025-01-18,2025-02-05,104551.6,96616.1,-0.075900,-7935.5,18,long_exit_signal,104102.721569,104092.669027,4296.650000,97647.694027,102885.733489,103502.375882,4443.435714,96837.222311
1,2,Long,2025-05-13,2025-11-03,104127.7,106584.3,0.023592,2456.6,174,long_exit_signal,93018.173195,92871.039113,2837.321429,88615.056971,112454.844152,112100.797732,3459.464286,106911.601304


## 5. Visualisasi

In [8]:

fig, ax = plt.subplots(figsize=(14, 6))
close_prices = data[PRICE_COLUMN]
ax.plot(close_prices.index, close_prices, label='Close', color='C0')

if 'ema_fast' in signals.columns:
    ax.plot(signals.index, signals['ema_fast'], label='EMA Fast', color='C1', linewidth=1.2)
if 'ema_slow' in signals.columns:
    ax.plot(signals.index, signals['ema_slow'], label='EMA Slow', color='C2', linewidth=1.2)
if 'atr_trailing_stop' in signals.columns:
    ax.plot(signals.index, signals['atr_trailing_stop'], label='ATR Stop', color='C3', linestyle='--')

long_entries = signals['long_entry'].fillna(False)
long_exits = signals['long_exit'].fillna(False)
ax.scatter(close_prices.index[long_entries], close_prices[long_entries], marker='^', color='green', label='Long Entry')
ax.scatter(close_prices.index[long_exits], close_prices[long_exits], marker='v', color='red', label='Long Exit')

ax.set_title(f'{ASSET_SYMBOL} Close dengan Sinyal {STRATEGY_NAME}')
ax.set_ylabel('Harga')
ax.legend()
plt.show()


In [9]:

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(outputs.results.index, outputs.results['equity_curve'], color='C4', label='Equity Curve')
ax.set_title('Equity Curve Strategi')
ax.set_ylabel('Notional')
ax.legend()
plt.show()

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(outputs.trades['pnl_pct'], bins=20, color='C0', alpha=0.7)
ax.set_title('Distribusi PnL per Trade')
ax.set_xlabel('PnL %')
ax.set_ylabel('Frekuensi')
plt.show()


## 6. Analisis trade kalah & investigasi

In [10]:

losing_trades = outputs.trades[outputs.trades['pnl_pct'] < 0]
print(f'Jumlah trade kalah: {len(losing_trades)} dari total {len(outputs.trades)}')
if not losing_trades.empty:
    print('Statistik ringkas trade kalah:')
    display(losing_trades[['pnl_pct', 'bars_held', 'exit_reason']].describe(include='all'))

    grouped = losing_trades.groupby('exit_reason')['pnl_pct'].agg(['count', 'mean', 'min'])
    print('Kinerja berdasarkan alasan exit:')
    display(grouped)

    context_cols = [col for col in outputs.trades.columns if col.startswith('entry_') or col.startswith('exit_')]
    if context_cols:
        print('Contoh konteks indikator pada trade kalah:')
        display(losing_trades[['trade_id', 'direction'] + context_cols].head())
else:
    print('Tidak ada trade kalah pada konfigurasi ini.')


Jumlah trade kalah: 1 dari total 2
Statistik ringkas trade kalah:


,pnl_pct,bars_held,exit_reason
count,1.0000,1.0,1
unique,NaN,NaN,1
top,NaN,NaN,long_exit_signal
freq,NaN,NaN,1
mean,-0.0759,18.0,NaN
std,NaN,NaN,NaN
min,-0.0759,18.0,NaN
25%,-0.0759,18.0,NaN
50%,-0.0759,18.0,NaN
75%,-0.0759,18.0,NaN


Kinerja berdasarkan alasan exit:


,count,mean,min
exit_reason,,,
long_exit_signal,1,-0.0759,-0.0759


Contoh konteks indikator pada trade kalah:


,trade_id,direction,entry_time,exit_time,entry_price,exit_price,exit_reason,entry_ema_fast,entry_ema_slow,entry_atr,entry_atr_trailing_stop,exit_ema_fast,exit_ema_slow,exit_atr,exit_atr_trailing_stop
0,1,Long,2025-01-18,2025-02-05,104551.6,96616.1,long_exit_signal,104102.721569,104092.669027,4296.65,97647.694027,102885.733489,103502.375882,4443.435714,96837.222311


## 7. Eksplor optimasi parameter

In [11]:

def sweep_parameters(base_kwargs: Dict[str, float], grid: Dict[str, Iterable[float]]):
    if not grid:
        return []
    results = []
    for multiplier in grid.get('atr_multiplier', []):
        kwargs = dict(base_kwargs)
        kwargs['atr_multiplier'] = multiplier
        strat = get_strategy(STRATEGY_NAME, **kwargs)
        sig = strat.generate_signals(data)
        backtest = backtester.run(sig)
        results.append({
            'atr_multiplier': multiplier,
            'total_return': backtest.metrics.get('total_return', np.nan),
            'sharpe_ratio': backtest.metrics.get('sharpe_ratio', np.nan),
            'trades': backtest.trade_summary['total_trades'],
        })
    return results

opt_results = sweep_parameters(STRATEGY_KWARGS, OPTIMISATION_GRID)
if opt_results:
    optimisation_df = pd.DataFrame(opt_results).sort_values('sharpe_ratio', ascending=False)
    display(optimisation_df)
else:
    print('Grid optimasi kosong – lewati tahap ini.')


,atr_multiplier,total_return,sharpe_ratio,trades
0,1.0,-0.021902,-0.011235,2
1,1.5,-0.054099,-0.095162,2
2,2.0,-0.107116,-0.295087,2



## 8. Dokumentasi & reusable structure

- Ganti `DATA_FILE` untuk menguji dataset TradingView lain yang memakai format serupa.
- Tambah atau modifikasi strategi di `src/strategy_backtest/strategies/`, lalu daftarkan nama modul di `registry.py`.
- Notebook ini mengandalkan `SignalBacktester` untuk menghasilkan trade log, metrik QF-Lib, serta visualisasi standar sehingga langkah analisis tetap konsisten antar strategi.
- Simpan variasi eksperimen (mis. hasil optimasi) ke file CSV tambahan bila diperlukan untuk laporan lebih lanjut.
